In [1]:
import numpy as np
import fpGrowth

In [2]:
rootNode = fpGrowth.treeNode('pyramid',9, None)
rootNode.children['eye']=fpGrowth.treeNode('eye', 13, None)
rootNode.disp()

  pyramid   9
   eye   13


In [3]:
rootNode.children['phoenix']=fpGrowth.treeNode('phoenix', 3, None)
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


In [4]:
simpDat = fpGrowth.loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [5]:
initSet = fpGrowth.createInitSet(simpDat) # 为什么要转成frozenset？
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [6]:
myFPtree, myHeaderTab = fpGrowth.createTree(initSet, 3)

item count =  {'p': 2, 'r': 3, 'h': 1, 'j': 1, 'z': 5, 'w': 1, 'v': 1, 'y': 3, 'u': 1, 's': 3, 'x': 4, 't': 3, 'o': 1, 'n': 1, 'q': 2, 'e': 1, 'm': 1}
headerTable =  {'r': [3, None], 'z': [5, None], 'y': [3, None], 's': [3, None], 'x': [4, None], 't': [3, None]}


In [7]:
myFPtree.disp() # 对相同Frequent的item，排序结果与书上不同，导致树与书上不同

  Null Set   1
   z   5
    r   1
    x   3
     y   2
      s   2
       t   2
     r   1
      y   1
       t   1
   x   1
    r   1
     s   1


In [8]:
myHeaderTab

{'r': [3, <fpGrowth.treeNode at 0x22a158060f0>],
 'z': [5, <fpGrowth.treeNode at 0x22a15806198>],
 'y': [3, <fpGrowth.treeNode at 0x22a15806240>],
 's': [3, <fpGrowth.treeNode at 0x22a15806278>],
 'x': [4, <fpGrowth.treeNode at 0x22a15806208>],
 't': [3, <fpGrowth.treeNode at 0x22a15806320>]}

In [9]:
# myHeaderTab是个set
# set的key是一个item，set的value是个list
# list的第0个元素是这个item的count，第1个元素是这个item的第一个node的link
fpGrowth.findPrefixPath(myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [10]:
fpGrowth.findPrefixPath(myHeaderTab['z'][1])

{}

In [11]:
fpGrowth.findPrefixPath(myHeaderTab['r'][1]) #生成的树不同，导致path不同

{frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'x', 'z'}): 1}

In [12]:
freqItems = []
fpGrowth.mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for:  {'s'}
  Null Set   1
   x   3
conditional tree for:  {'t'}
  Null Set   1
   y   2
    z   2
     x   2
   z   1
    y   1
     x   1
conditional tree for:  {'x'}
  Null Set   1
   z   3
conditional tree for:  {'y'}
  Null Set   1
   z   3
    x   3
conditional tree for:  {'y', 'x'}
  Null Set   1
   z   3


In [13]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'y'},
 {'t', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'y', 'z'},
 {'z'}]

# 12.4的数据访问不了

In [14]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]

In [15]:
myFPtree, myHeaderTab = fpGrowth.createTree(parsedDat, 100000)

item count =  {'1': 197522, '2': 42927, '3': 450031, '4': 78097, '5': 5930, '6': 601374, '7': 86898, '8': 26, '9': 1519, '10': 294, '11': 364065, '12': 5868, '13': 690, '14': 11607, '15': 58, '16': 316, '17': 2437, '18': 5917, '19': 1166, '20': 4504, '21': 8763, '22': 21, '23': 1084, '24': 2330, '25': 12668, '26': 18, '27': 72134, '28': 10111, '29': 1091, '30': 27, '31': 138, '32': 11069, '33': 299, '34': 121, '35': 133, '36': 619, '37': 3985, '38': 1945, '39': 3876, '40': 24082, '41': 4933, '42': 818, '43': 2877, '44': 2577, '45': 6581, '46': 8827, '47': 1334, '48': 680, '49': 12702, '50': 1445, '51': 2021, '52': 1967, '53': 40, '54': 1960, '55': 65412, '56': 15752, '57': 1179, '58': 2413, '59': 864, '60': 823, '61': 856, '62': 2853, '63': 250, '64': 48262, '65': 30, '66': 472, '67': 46, '68': 9, '69': 22631, '70': 1773, '71': 6643, '72': 1967, '73': 18749, '74': 221, '75': 11, '76': 580, '77': 43454, '78': 2656, '79': 70, '80': 105, '81': 11, '82': 1964, '83': 32212, '84': 92, '85': 

In [16]:
myFreqList = []
fpGrowth.mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

conditional tree for:  {'1'}
  Null Set   1
   6   132113
conditional tree for:  {'11'}
  Null Set   1
   6   324013
    3   143682
   3   17604
conditional tree for:  {'11', '3'}
  Null Set   1
   6   143682
conditional tree for:  {'3'}
  Null Set   1
   6   265180


In [17]:
len(myFreqList)

9

In [18]:
myFreqList

[{'1'},
 {'1', '6'},
 {'11'},
 {'11', '3'},
 {'11', '3', '6'},
 {'11', '6'},
 {'3'},
 {'3', '6'},
 {'6'}]